In [4]:
import os
import shutil

# Rutas de los datasets y el destino
datasets = ["./datasetCreado", "./datasetGrandeRflow", "./datasetPequeñoRflow"] 
output_dir = "./datasetUnificado" 
images_output_dir = os.path.join(output_dir, "images")
labels_output_dir = os.path.join(output_dir, "labels")

os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(labels_output_dir, exist_ok=True)

def procesar_estructura(dataset_path, images_output_dir, labels_output_dir, image_count):
    estructuras = [
        ["train/images", "train/labels"],
        ["test/images", "test/labels"],
        ["valid/images", "valid/labels"],
        ["images/train", "labels/train"],
        ["images/test", "labels/test"],
        ["images/val", "labels/val"]
    ]

    for estructura in estructuras:
        images_dir = os.path.join(dataset_path, estructura[0])
        labels_dir = os.path.join(dataset_path, estructura[1])

        if os.path.exists(images_dir) and os.path.exists(labels_dir):
            for file_name in os.listdir(images_dir):
                if file_name.endswith(('.jpg', '.png', '.jpeg')):
                    base_name = os.path.splitext(file_name)[0]

                    # Rutas de origen
                    img_src = os.path.join(images_dir, file_name)
                    lbl_src = os.path.join(labels_dir, f"{base_name}.txt")

                    # Nombres únicos para los archivos destino
                    new_img_name = f"{image_count:06d}.jpg"
                    new_lbl_name = f"{image_count:06d}.txt"

                    # Copiar imagen
                    shutil.copy(img_src, os.path.join(images_output_dir, new_img_name))
                    # Copiar etiqueta si existe
                    if os.path.exists(lbl_src):
                        shutil.copy(lbl_src, os.path.join(labels_output_dir, new_lbl_name))

                    image_count += 1
    return image_count

# Combina todos los datasets
image_count = 0
for dataset in datasets:
    image_count = procesar_estructura(dataset, images_output_dir, labels_output_dir, image_count)

print(f"Unificación completa. Total de imágenes procesadas: {image_count}")


Unificación completa. Total de imágenes procesadas: 1266


In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split

def create_dirs(base_path, categories):
    for category in categories:
        os.makedirs(os.path.join(base_path, category), exist_ok=True)

def move_files(files, src_dir, dest_dir):
    for file in files:
        shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))

def split_images(base_path, test_size=0.2, val_size=0.1):
    images_path = os.path.join(base_path, "images")

    # Create the new directories
    create_dirs(images_path, ["train", "test", "val"])

    # Get all image files
    image_files = [f for f in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, f))]

    # Split the dataset
    train_images, test_images = train_test_split(image_files, test_size=test_size, random_state=42)
    train_images, val_images = train_test_split(train_images, test_size=val_size / (1 - test_size), random_state=42)

    # Move images
    move_files(test_images, images_path, os.path.join(images_path, "test"))
    move_files(train_images, images_path, os.path.join(images_path, "train"))
    move_files(val_images, images_path, os.path.join(images_path, "val"))

if __name__ == "__main__":
    base_dataset_path = "C:/Users/Guille/Documents/TFG/datasetUnificado"  # Cambia esto a la ruta de tu dataset
    split_images(base_dataset_path)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [8]:
import os
import shutil

def create_dirs(base_path, categories):
    for category in categories:
        os.makedirs(os.path.join(base_path, category), exist_ok=True)

def move_files(files, src_dir, dest_dir):
    for file in files:
        shutil.move(os.path.join(src_dir, file), os.path.join(dest_dir, file))

def split_labels(base_path, labels_ext=".txt"):
    images_path = os.path.join(base_path, "images")
    labels_path = os.path.join(base_path, "labels")

    # Create the new directories
    create_dirs(labels_path, ["train", "test", "val"])

    # Get lists of files already moved to each split
    for subfolder in ["train", "test", "val"]:
        image_files = [f for f in os.listdir(os.path.join(images_path, subfolder)) if os.path.isfile(os.path.join(images_path, subfolder, f))]
        label_files = [f.replace('.jpg', labels_ext) for f in image_files]

        # Move corresponding labels
        move_files(label_files, labels_path, os.path.join(labels_path, subfolder))

if __name__ == "__main__":
    base_dataset_path = "C:/Users/Guille/Documents/TFG/datasetUnificado"  # Cambia esto a la ruta de tu dataset
    split_labels(base_dataset_path)
